In [13]:
import pandas as pd

df=pd.read_csv('2025-04-19 - PEN-EXG.CSV', encoding='latin-1', sep=';')

df['Última Atualização'] = pd.to_datetime(df['Última Atualização'], format='%d/%m/%Y %H:%M', dayfirst=True, errors='coerce')
df['Data de Entrada do Requerimento'] = pd.to_datetime(df['Data de Entrada do Requerimento'], format='%d/%m/%Y %H:%M', dayfirst=True, errors='coerce')
df['DER_Year'] = df['Data de Entrada do Requerimento'].dt.year
df['DER_Month'] = df['Data de Entrada do Requerimento'].dt.month
df['DER_Day'] = df['Data de Entrada do Requerimento'].dt.day
df['DER_Hour'] = df['Data de Entrada do Requerimento'].dt.hour
df = df.drop(columns=['Data de Entrada do Requerimento'])
df['Ultima_Year'] = df['Última Atualização'].dt.year
df['Ultima_Month'] = df['Última Atualização'].dt.month
df['Ultima_Day'] = df['Última Atualização'].dt.day
df['Ultima_Hour'] = df['Última Atualização'].dt.hour
df = df.drop(columns=['Última Atualização'])

excluded_services = ['CPF Cancelado na RFB',
                     'CPF Suspenso na RFB',
                     'Qualificação da Folha de Pagamento_SVCBEN/Painel QDBEN_Inconsistências de CPF na base da RFB']
filtered_df = df[~df['Serviço'].isin(excluded_services)]

pivot_table = filtered_df.pivot_table(index=['Serviço','Responsáveis'],
                             columns='DER_Year',
                             aggfunc='size',
                             fill_value=0)

filtered_pivot_table = pivot_table[[2021, 2022, 2023]]
filtered_pivot_table = filtered_pivot_table[(filtered_pivot_table.T != 0).any()]
filtered_pivot_table
# Include data for January to June 2024
for month in range(1, 7):  # Loop through months 1 to 6 (Jan-Jun)
    col_name = f'2024_{month:02}'  # Create column name like '2024_01', '2024_02', etc.
    filtered_pivot_table[col_name] = df[
        (df['DER_Year'] == 2024) & (df['DER_Month'] == month)
    ].groupby(['Serviço','Responsáveis']) \
        .size() \
        .reindex(filtered_pivot_table.index, fill_value=0)

filtered_pivot_table
styled_pivot_table = filtered_pivot_table.style \
    .background_gradient(cmap='Blues') \
    .set_properties(**{'text-align': 'center'}) \
    .format("{:.0f}")

styled_pivot_table